# $Getting$ $Data$ $(Option$ $Chains)$ $from$ $Yahoo$ $Finance.$
# $(Step-by-step$ $Guide).$

For such purpose will ned the following python libraries:

In [1]:
import pandas as pd
import datetime
import yfinance as yf

For example let's consider stock of JPMorgan Chase & Co. as an underlying asset.

In [2]:
underlying = yf.Ticker("JPM")

The ".options" will provide us with dates options on JPM expire, meanwhile ".option_chain" will give us of puts and calls referring the following way:
(Also, we'll pick date 2025/01/17 as an example.

In [3]:
expirations_dates = underlying.options
expirations_dates

('2023-02-03',
 '2023-02-10',
 '2023-02-17',
 '2023-02-24',
 '2023-03-03',
 '2023-03-10',
 '2023-03-17',
 '2023-04-21',
 '2023-06-16',
 '2023-07-21',
 '2023-09-15',
 '2024-01-19',
 '2024-06-21',
 '2025-01-17')

In [4]:
date  = '2025-01-17'

In [5]:
opt = underlying.option_chain(date)

In [6]:
opt.calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,JPM250117C00055000,2023-01-18 19:26:25+00:00,55.0,82.85,0.0,0.00,0.0,0.0,2,0,0.000010,True,REGULAR,USD
1,JPM250117C00060000,2023-01-31 19:10:03+00:00,60.0,80.00,0.0,0.00,0.0,0.0,1,0,0.000010,True,REGULAR,USD
2,JPM250117C00065000,2023-01-27 17:54:04+00:00,65.0,76.50,0.0,0.00,0.0,0.0,1,0,0.000010,True,REGULAR,USD
3,JPM250117C00070000,2023-01-18 19:23:50+00:00,70.0,69.00,0.0,0.00,0.0,0.0,2,0,0.000010,True,REGULAR,USD
4,JPM250117C00075000,2023-01-03 14:35:42+00:00,75.0,62.88,64.8,68.75,0.0,0.0,130,169,0.431646,True,REGULAR,USD


In [7]:
opt.puts.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,JPM250117P00055000,2023-02-01 19:27:46+00:00,55.0,0.96,0.0,0.0,0.0,0.0,8,0,0.125009,False,REGULAR,USD
1,JPM250117P00060000,2023-02-01 19:30:13+00:00,60.0,1.18,0.0,0.0,0.0,0.0,8,0,0.125009,False,REGULAR,USD
2,JPM250117P00065000,2023-01-30 18:28:09+00:00,65.0,1.47,0.0,0.0,0.0,0.0,2,0,0.125009,False,REGULAR,USD
3,JPM250117P00070000,2023-01-24 20:05:32+00:00,70.0,1.72,0.0,0.0,0.0,0.0,1,0,0.125009,False,REGULAR,USD
4,JPM250117P00075000,2023-02-01 19:21:27+00:00,75.0,2.11,0.0,0.0,0.0,0.0,4,0,0.125009,False,REGULAR,USD


In [8]:
def yf_get_chains(ticker):
    """
    Loads option chains data for a particular ticker from Yahoo Finance
    """

    underlying = yf.Ticker(ticker)  ## addressing the underlying asset of an options
    expiration_dates = underlying.options  ## and dates of expiration

    opt_data = pd.DataFrame()  ## constructing the dataframe we will return

    for ed in expiration_dates:
        opt = underlying.option_chain(ed)  ## getting data for a specific date

        calls, puts = opt.calls, opt.puts  ## call and puts
        calls['optionType'], puts['optionType'] = 'call', 'puts'  ## adding types of options to the df

        ed_chain = pd.concat([calls, puts])
        ed_chain['expDate'] = pd.to_datetime(ed) + pd.DateOffset(hours=23, minutes=59, seconds=59)
        ## adding date of expiration

        opt_data = pd.concat([opt_data, ed_chain])  ## adding data to the main df

    opt_data['daysToExp'] = (opt_data.expDate - datetime.datetime.today()).dt.days + 1
    ## additionally let's compute days until expiration date

    return opt_data

In [9]:
ticker = 'JPM'

In [10]:
jpm_yf = yf_get_chains(ticker)
jpm_yf.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expDate,daysToExp
0,JPM230203C00100000,2023-01-03 19:38:02+00:00,100.0,34.05,38.80,39.00,0.0,0.0,100.0,0,2.718753,True,REGULAR,USD,call,2023-02-03 23:59:59,1
1,JPM230203C00105000,2023-01-03 18:15:30+00:00,105.0,29.37,33.60,34.25,0.0,0.0,1.0,0,3.054690,True,REGULAR,USD,call,2023-02-03 23:59:59,1
2,JPM230203C00110000,2023-02-02 16:27:29+00:00,110.0,29.74,0.00,0.00,0.0,0.0,10.0,0,0.000010,True,REGULAR,USD,call,2023-02-03 23:59:59,1
3,JPM230203C00115000,2022-12-30 17:53:27+00:00,115.0,19.00,25.15,25.60,0.0,0.0,6.0,0,3.248049,True,REGULAR,USD,call,2023-02-03 23:59:59,1
4,JPM230203C00117000,2023-01-27 16:08:18+00:00,117.0,22.90,0.00,0.00,0.0,0.0,7.0,0,0.000010,True,REGULAR,USD,call,2023-02-03 23:59:59,1


In [11]:
print(len(expirations_dates))
print(jpm_yf.expDate.unique().shape[0])

14
14


## Here we go, dataset is constructed and ready to use.
Please, notice that for dynamic update of days until expiration (column "daysToExp" one has to run function again. Thus it might become outdated as time passes.
